# Optical properties of a lightsail made of N slabs of FeS2

Slabs of FeS2 have a thickness of $d_1$ and are separated by layers of ideal aerogel (dielectric constant equal to the one of the vacuum and 0 density) of thickness $d_2$. 

Let a stack of $N$ slabs separated by aerogel layers.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def computeRT(d1,d2,N,n,phi0,omega) : 
    
    Rs = abs(rs)**2
    Rp = abs(rp)**2
    #final results
    R = 0.5*Rs+0.5*Rp
    T = 0.5*Ts+0.5*T
    return (R,T)